In [7]:
import numpy as np, pandas as pd, warnings, os, openai, json
from tqdm.auto import tqdm
from openai import OpenAI
warnings.filterwarnings('ignore')
import requests
from typing import List, Tuple, Union
from langchain_ollama.embeddings import OllamaEmbeddings
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from haversine import haversine, Unit
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda
from langchain.memory import ConversationBufferMemory
# from langchain_ollama.chat_models import ChatOpenAI
# from langchain.schema import Sys
from langgraph.graph import StateGraph, END

In [11]:
key = open('../../../../api_key.txt','r')
api_key = key.read()
openai.api_key = api_key

base_ = open('../../../../base_url.txt','r')
base_url = base_.read()
# openai.api_key = api_key

In [ ]:
llm = ChatOllama(model = 'gemma3:12b',
                #  base_url=base_url,
                
                 )


# GPT.ver

In [14]:
def invoke(
           prompt,
           api_key=api_key,
           model="gpt-4o",
           temperature=0.7):
    client = OpenAI(api_key=api_key)

    # 메시지 구성
    messages = [
        # {"role": "system", "content": prompt},
        {"role": "user", "content": prompt}
    ]
    # 스트리밍 요청
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        # stream=True  # ⭐ 핵심 옵션!
    )

    # generator 반환 (chunk 단위 텍스트 출력)
    return stream

In [17]:
prompt = f"""
                너는 사용자의 질문에 똑똑하고 정확하게 대답해주는 도우미야.
                포카라에 대해서 설명해줘
                """

In [27]:
response = invoke(prompt).choices[0].message.content